<a href="https://colab.research.google.com/github/chewzzz1014/fyp/blob/master/ner/src/train_ner_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train NER Models

In [2]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!mkdir spacy_ner_data

In [13]:
import json
import random
from sklearn.model_selection import train_test_split
import spacy
from spacy.tokens import DocBin

# Load JSON data
with open('/content/drive/MyDrive/FYP/Implementation/Resume Dataset/40_resumes_annotated.json', "r") as f:
    data = json.load(f)

def remove_overlapping_entities(entities):
    """Remove overlapping entities from the list."""
    entities = sorted(entities, key=lambda x: x[0])  # Sort by start position
    non_overlapping = []
    last_end = -1
    for start, end, label in entities:
        if start >= last_end:  # Only add if there's no overlap with the previous entity
            non_overlapping.append((start, end, label))
            last_end = end
    return non_overlapping

# Function to convert JSON data to Spacy's DocBin format
def convert_to_spacy_format(data):
    nlp = spacy.blank("en")  # Load a blank Spacy model
    doc_bin = DocBin()  # Container for our docs

    for item in data:
        text = item['data']['Text']  # Full document text
        entities = []

        for annotation in item['annotations'][0]['result']:
            start = annotation['value']['start']
            end = annotation['value']['end']
            label = annotation['value']['labels'][0]  # Entity label
            entities.append((start, end, label))

        entities = remove_overlapping_entities(entities)  # Remove overlapping entities
        # Create a Spacy doc and add entities to it
        doc = nlp.make_doc(text)
        spans = [doc.char_span(start, end, label=label) for start, end, label in entities]
        # Filter out None spans if Spacy can't align the character indices with tokens
        spans = [span for span in spans if span is not None]
        doc.ents = spans  # Assign entities to the doc
        doc_bin.add(doc)

    return doc_bin

# Split data into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Convert train and test sets to Spacy format
train_doc_bin = convert_to_spacy_format(train_data)
test_doc_bin = convert_to_spacy_format(test_data)

# Save the train and test data to .spacy files
train_doc_bin.to_disk("spacy_ner_data/train_data.spacy")
test_doc_bin.to_disk("spacy_ner_data/test_data.spacy")

## Spacy NER

In [5]:
# create base_config.cfg and paste the config generated from spacy widget
# update train and test file path
!touch base_config.cfg

In [7]:
# generate config.cfg from base_config.cfg
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [19]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 801.1 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [15]:
# train model using hyperparameters set in config.cfg
# trained model in output/ dir
!python -m spacy train config.cfg --output ./output

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.0005
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    238.65    0.00    0.00    0.00    0.00
  3     100       2129.88   9813.94   18.94   21.25   17.09    0.19
  6     200        433.86   5971.17   27.32   32.56   23.53    0.27
  9     300        174.71   4187.37   32.45   28.57   37.54    0.32
 12     400        655.81   3933.13   30.71   26.71   36.13    0.31
 15     500        834.10   2505.04   27.80   19.36   49.30    0.28
 18     600        112.18   1961.11   31.86   27.24   38.38    0.32
 21     700         97.26   1453.16   34.12   30.72   38.38    0.34
 25     800        109.06   12

In [17]:
# evaluate trained model performance
# store output and visualization into result/ dir
!python -m spacy evaluate output/model-best spacy_ner_data/test_data.spacy -dp output

ℹ Using CPU

================================== Results ==================================

TOK     100.00
NER P   30.00 
NER R   40.34 
NER F   34.41 
SPEED   2476  


=============================== NER (per type) ===============================

                 P        R        F
NAME        100.00    62.50    76.92
PHONE       100.00   100.00   100.00
SKILL        24.81    36.84    29.65
WORK PER     48.39    75.00    58.82
COMPANY      14.29     5.88     8.33
JOB          53.85    33.33    41.18
STUDY PER    50.00    50.00    50.00
DEG          33.33    50.00    40.00
UNI          55.56    71.43    62.50
LOC         100.00    75.00    85.71

<IPython.core.display.HTML object>
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/spacy/__main__.py

In [18]:
# make prediction
import spacy
resume_text = 'dot net developer robert smith phone 123 456 78 99 email infoqwikresumecom website wwwqwikresumecom linkedin linkedincomqwikresume address 1737 marshville road alabama objective dot net developer seven years experience design development webbased windows based applications using net technologies hands experience phases software development life cycle sdlc like requirement gathering analysis architectural detail design documentation development testing implementation using agile methodologies like scrum xp test driven environment skills programming languages net technologies c sql plsql web forms win forms web technologies scripting html dhtml css xmlangular jsbootstrap ajax toolkit jquery javascript telerikkendo ui database ms sql server operating systems windows 8 packages ms office amp visio ms frontpage iis version control tools git hub vss tfs methodologies agile oops scrum soa reporting crystal reports net ms sql server reporting services ssrs work experience dot net developer charter communications june 2015 present assisting developing architectural design functional specifications involving analysis designing coding implementation application developing dynamic web page implemented creatively implemented design requirements using client side scripting language technologies assisting agile software development management activities respond unpredictability iterative sprints designing developing web application migrating project mvc architecture using mvc 3 separate internal representations information involved developing telerik kendo ul controls building application enabling focus value generating development tasks involving development presentation logic gui aspnet pages dot net developer abc corp 2011 2015 coded updated maintained computer programs assisted developers prepare high level technical design documents performed code enhancements assist performance analysis provided technical guidance programming standards team trained users team coordinated client amp offshore tearn meet project objectives participated backlog grooming meeting work client remove barriers team worked individual developer also manage offshore team free resume template copyright qwikresumecom usage guidelines'
nlp = spacy.load("output/model-best")
doc = nlp(resume_text)

print(doc.ents)

for ent in doc.ents:
    print(f"{ent.text}: {ent.label_}")

(robert smith, 123 456 78 99, infoqwikresumecom, alabama, agile, scrum, c, sql, plsql, scripting, html, css, jquery, ui, database, ms sql, windows, git, tfs, agile, oops, scrum, soa, dot net developer, june 2015 present assisting, mvc, mvc, gui, aspnet, abc corp, 2011 2015)
robert smith: NAME
123 456 78 99: PHONE
infoqwikresumecom: EMAIL
alabama: LOC
agile: SKILL
scrum: SKILL
c: SKILL
sql: SKILL
plsql: SKILL
scripting: SKILL
html: SKILL
css: SKILL
jquery: SKILL
ui: SKILL
database: SKILL
ms sql: SKILL
windows: SKILL
git: SKILL
tfs: SKILL
agile: SKILL
oops: SKILL
scrum: SKILL
soa: SKILL
dot net developer: JOB
june 2015 present assisting: WORK PER
mvc: SKILL
mvc: SKILL
gui: SKILL
aspnet: SKILL
abc corp: COMPANY
2011 2015: WORK PER


In [19]:
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

In [ ]:
# download trained model

## Flair NER

In [8]:
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 6.0 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=b12b05a347328d7be79baf7a5dcb17dd6e0e31ea1eb0789f957902e3f5070f30
  Stored in directory: /root/.cach

In [9]:
import spacy
from spacy.tokens import DocBin
import os

def convert_spacy_to_flair(input_file, output_file):
    """
    Convert SpaCy binary format to Flair's CoNLL format.

    Args:
        input_file (str): Path to SpaCy binary file (.spacy)
        output_file (str): Path to output file for Flair format
    """
    # Load spaCy model
    nlp = spacy.blank("en")

    # Load the DocBin
    doc_bin = DocBin().from_disk(input_file)
    docs = list(doc_bin.get_docs(nlp.vocab))

    with open(output_file, 'w', encoding='utf-8') as f:
        for doc in docs:
            tokens = [(t.text, t.ent_iob_, t.ent_type_) for t in doc]

            # Write tokens in CoNLL format
            for token in tokens:
                text, iob, ent_type = token

                # Convert spaCy IOB to CoNLL format
                if iob == 'O':
                    tag = 'O'
                else:
                    tag = f'{iob}-{ent_type}' if ent_type else 'O'

                # Write line: token and NER tag
                f.write(f'{text} {tag}\n')

            # Empty line between sentences
            f.write('\n')

def convert_spacy_json_to_flair(input_file, output_file):
    """
    Convert SpaCy JSON format to Flair's CoNLL format.

    Args:
        input_file (str): Path to JSON file with SpaCy annotations
        output_file (str): Path to output file for Flair format
    """
    import json

    nlp = spacy.blank("en")

    with open(input_file, 'r', encoding='utf-8') as f:
        training_data = json.load(f)

    with open(output_file, 'w', encoding='utf-8') as f:
        for example in training_data:
            text = example['text']
            ents = example.get('entities', [])

            # Create a spaCy doc
            doc = nlp(text)

            # Add entities to doc
            spans = []
            for start, end, label in ents:
                span = doc.char_span(start, end, label=label)
                if span is not None:
                    spans.append(span)
            doc.ents = spans

            # Convert to CoNLL format
            tokens = [(t.text, t.ent_iob_, t.ent_type_) for t in doc]

            for token in tokens:
                text, iob, ent_type = token
                if iob == 'O':
                    tag = 'O'
                else:
                    tag = f'{iob}-{ent_type}' if ent_type else 'O'
                f.write(f'{text} {tag}\n')

            f.write('\n')

# Example usage for JSON format
flair_train_json = "flair_train.txt"
flair_test_json = "flair_test.txt"

convert_spacy_to_flair('/content/spacy_ner_data/train_data.spacy', flair_train_json)
convert_spacy_to_flair('/content/spacy_ner_data/test_data.spacy', flair_test_json)

In [21]:
import spacy
from spacy.training import Corpus

# !python -m spacy download de_core_news_sm
nlp = spacy.load("en_core_web_lg")
corpus = Corpus("/content/spacy_ner_data/train_data.spacy")

data = corpus(nlp)

# Flair supports BIO and BIOES, see https://github.com/flairNLP/flair/issues/875
def rename_biluo_to_bioes(old_tag):
    new_tag = ""
    try:
        if old_tag.startswith("L"):
            new_tag = "E" + old_tag[1:]
        elif old_tag.startswith("U"):
            new_tag = "S" + old_tag[1:]
        else:
            new_tag = old_tag
    except:
        pass
    return new_tag


def generate_corpus():
    corpus = []
    n_ex = 0
    for example in data:
        n_ex += 1
        text = example.text
        doc = nlp(text)
        tags = example.get_aligned_ner()
        # Check if it's an empty list of NER tags.
        if None in tags:
            pass
        else:
            new_tags = [rename_biluo_to_bioes(tag) for tag in tags]
            for token, tag in zip(doc,new_tags):
                row = token.text +' '+ token.pos_ +' ' +tag + '\n'
                corpus.append(row)
            corpus.append('\n')
    return corpus

def write_file(filepath):
    with open(filepath, 'w', encoding='utf-8') as f:
        corpus = generate_corpus()
        f.writelines(corpus)

def main():
    write_file('flair_train.txt')

main()

In [22]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# Define columns for CoNLL (0: word, 1: label)
columns = {0: 'text', 1: 'ner'}

# Set data folder and file names
data_folder = './'
train_file = 'flair_train.txt'
test_file = 'flair_test.txt'

# Load the corpus
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file=train_file,
                              test_file=test_file)

2024-10-30 09:33:06,117 Reading data from .
2024-10-30 09:33:06,120 Train: flair_train.txt
2024-10-30 09:33:06,121 Dev: None
2024-10-30 09:33:06,123 Test: flair_test.txt
2024-10-30 09:33:07,563 No dev split found. Using 10% (i.e. 3 samples) of the train split as dev data


In [23]:
# create NER tagger
from flair.embeddings import WordEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger

embeddings = StackedEmbeddings([
                WordEmbeddings('glove'),
                WordEmbeddings('en-crawl')
            ])

tagger = SequenceTagger(hidden_size=256,
                         embeddings=embeddings,
                         tag_dictionary=corpus.make_label_dictionary(tag_type="ner"),
                         tag_type='ner',
                         use_crf=True)

2024-10-30 09:33:39,809 SequenceTagger predicts: Dictionary with 19 tags: O, PROPN, NUM, ADJ, NOUN, VERB, X, ADP, PUNCT, ADV, SCONJ, AUX, INTJ, SYM, DET, PRON, CCONJ, <START>, <STOP>


<ipython-input-23-687232df04ba>:12: DeprecationWarning: Call to deprecated method make_tag_dictionary. (Use 'make_label_dictionary' instead.) -- Deprecated since version 0.8.
  tag_dictionary=corpus.make_tag_dictionary(tag_type='ner'),


In [24]:
# train flair ner model
from flair.trainers import ModelTrainer
from flair.training_utils import EvaluationMetric

trainer = ModelTrainer(tagger, corpus)

trainer.train('flair_output/',
               learning_rate=0.1,
               mini_batch_size=32,
               max_epochs=10)

# trainer.train(
#      base_path='/flair_output/',
#      learning_rate=0.1,
#      mini_batch_size=32,
#      max_epochs=10,
#      patience=3,
#      embeddings_storage_mode='gpu',
#     #  checkpoint=True,
#      use_amp=True,  # Use mixed precision training
#      train_with_dev=False,
#     #  monitor_train=True,
#     #  monitor_test=True,
#      evaluation_metric=EvaluationMetric.MICRO_F1_SCORE,
# )

trainer.train(
    base_path='/flair_output/',
    learning_rate=0.001,
    mini_batch_size=32,
    max_epochs=100,
    patience=3,
    embeddings_storage_mode='gpu',
    use_amp=True,  # Use mixed precision training
    train_with_dev=False,
    main_evaluation_metric=EvaluationMetric.MICRO_F1_SCORE  # Set the evaluation metric here
)

2024-10-30 09:33:47,939 ----------------------------------------------------------------------------------------------------
2024-10-30 09:33:47,943 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
    (list_embedding_1): WordEmbeddings(
      'en-crawl'
      (embedding): Embedding(1000001, 300)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=400, out_features=400, bias=True)
  (rnn): LSTM(400, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=19, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)"
2024-10-30 09:33:47,946 ----------------------------------------------------------------------------------------------------
2024-10-30 09:33:47,949 Corpus: 29 train + 3 dev + 8 test sentences
2024-10-30 09:33:47,950 ------------------------------------

/usr/local/lib/python3.10/dist-packages/flair/trainers/trainer.py:499: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


2024-10-30 09:37:13,489 epoch 1 - iter 1/1 - loss 3.34127709 - time (sec): 205.50 - samples/sec: 62.37 - lr: 0.100000 - momentum: 0.000000
2024-10-30 09:37:13,493 ----------------------------------------------------------------------------------------------------
2024-10-30 09:37:13,496 EPOCH 1 done: loss 3.3413 - lr: 0.100000


100%|██████████| 1/1 [00:00<00:00,  3.62it/s]

2024-10-30 09:37:13,815 DEV : loss 2.9249958992004395 - f1-score (micro avg)  0.0142
2024-10-30 09:37:13,828  - 0 epochs without improvement
2024-10-30 09:37:13,832 saving best model


...

2024-10-30 10:10:00,145 
Results:
- F-score (micro) 0.4584
- F-score (macro) 0.0401
- Accuracy 0.4583

By class:
              precision    recall  f1-score   support

        NOUN     0.4615    0.9969    0.6309      2865
       PROPN     0.3684    0.0051    0.0100      1381
        VERB     0.0000    0.0000    0.0000      1120
         ADJ     0.0000    0.0000    0.0000       509
         NUM     0.0000    0.0000    0.0000       153
         ADV     0.0000    0.0000    0.0000        81
         ADP     0.0000    0.0000    0.0000        59
           X     0.0000    0.0000    0.0000        43
         AUX     0.0000    0.0000    0.0000        11
        INTJ     0.0000    0.0000    0.0000         8
       CCONJ     0.0000    0.0000    0.0000         6
       SCONJ     0.0000    0.0000    0.0000         3
        PRON     0.0000    0.0000    0.0000         3
       PUNCT     0.0000    0.0000    0.0000         3
         DET     0.0000    0.0000    0.0000         1
         SYM     0.000


/usr/local/lib/python3.10/dist-packages/flair/trainers/trainer.py:499: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


2024-10-30 10:10:00,262 Corpus: 29 train + 3 dev + 8 test sentences
2024-10-30 10:10:00,264 ----------------------------------------------------------------------------------------------------
2024-10-30 10:10:00,266 Train:  29 sentences
2024-10-30 10:10:00,267         (train_with_dev=False, train_with_test=False)
2024-10-30 10:10:00,268 ----------------------------------------------------------------------------------------------------
2024-10-30 10:10:00,269 Training Params:
2024-10-30 10:10:00,272  - learning_rate: "0.001" 
2024-10-30 10:10:00,273  - mini_batch_size: "32"
2024-10-30 10:10:00,274  - max_epochs: "100"
2024-10-30 10:10:00,275  - shuffle: "True"
2024-10-30 10:10:00,276 ----------------------------------------------------------------------------------------------------
2024-10-30 10:10:00,277 Plugins:
2024-10-30 10:10:00,278  - AnnealOnPlateau | patience: '3', anneal_factor: '0.5', min_learning_rate: '0.0001'
2024-10-30 10:10:00,279 --------------------------------------

100%|██████████| 1/1 [00:00<00:00,  2.39it/s]


TypeError: 'EvaluationMetric' object is not subscriptable

In [29]:
# evaluate model
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

# Load the trained model
model = SequenceTagger.load('/content/flair_output/best-model.pt')

# Evaluate the model on the test set
result = model.evaluate(corpus.test, gold_label_type='ner', mini_batch_size=32)

# Print the results
# print("Evaluation Loss:", eval_loss)
print(result.detailed_results)  # This will print the precision, recall, and F1-score per entity type

2024-10-30 10:22:17,612 SequenceTagger predicts: Dictionary with 19 tags: O, PROPN, NUM, ADJ, NOUN, VERB, X, ADP, PUNCT, ADV, SCONJ, AUX, INTJ, SYM, DET, PRON, CCONJ, <START>, <STOP>


100%|██████████| 1/1 [00:02<00:00,  2.74s/it]


Results:
- F-score (micro) 0.4584
- F-score (macro) 0.0401
- Accuracy 0.4583

By class:
              precision    recall  f1-score   support

        NOUN     0.4615    0.9969    0.6309      2865
       PROPN     0.3684    0.0051    0.0100      1381
        VERB     0.0000    0.0000    0.0000      1120
         ADJ     0.0000    0.0000    0.0000       509
         NUM     0.0000    0.0000    0.0000       153
         ADV     0.0000    0.0000    0.0000        81
         ADP     0.0000    0.0000    0.0000        59
           X     0.0000    0.0000    0.0000        43
         AUX     0.0000    0.0000    0.0000        11
        INTJ     0.0000    0.0000    0.0000         8
       CCONJ     0.0000    0.0000    0.0000         6
       SCONJ     0.0000    0.0000    0.0000         3
        PRON     0.0000    0.0000    0.0000         3
       PUNCT     0.0000    0.0000    0.0000         3
         DET     0.0000    0.0000    0.0000         1
         SYM     0.0000    0.0000    0.0000   

In [1]:
# make prediction
import flair
model = SequenceTagger.load('flair_output/final-model.pt')
resume_text = 'dot net developer robert smith phone 123 456 78 99 email infoqwikresumecom website wwwqwikresumecom linkedin linkedincomqwikresume address 1737 marshville road alabama objective dot net developer seven years experience design development webbased windows based applications using net technologies hands experience phases software development life cycle sdlc like requirement gathering analysis architectural detail design documentation development testing implementation using agile methodologies like scrum xp test driven environment skills programming languages net technologies c sql plsql web forms win forms web technologies scripting html dhtml css xmlangular jsbootstrap ajax toolkit jquery javascript telerikkendo ui database ms sql server operating systems windows 8 packages ms office amp visio ms frontpage iis version control tools git hub vss tfs methodologies agile oops scrum soa reporting crystal reports net ms sql server reporting services ssrs work experience dot net developer charter communications june 2015 present assisting developing architectural design functional specifications involving analysis designing coding implementation application developing dynamic web page implemented creatively implemented design requirements using client side scripting language technologies assisting agile software development management activities respond unpredictability iterative sprints designing developing web application migrating project mvc architecture using mvc 3 separate internal representations information involved developing telerik kendo ul controls building application enabling focus value generating development tasks involving development presentation logic gui aspnet pages dot net developer abc corp 2011 2015 coded updated maintained computer programs assisted developers prepare high level technical design documents performed code enhancements assist performance analysis provided technical guidance programming standards team trained users team coordinated client amp offshore tearn meet project objectives participated backlog grooming meeting work client remove barriers team worked individual developer also manage offshore team free resume template copyright qwikresumecom usage guidelines'
sentence = flair.data.Sentence(resume_text)

model.predict(sentence)

print(sentence.to_tagged_string())

NameError: name 'SequenceTagger' is not defined